<a href="https://colab.research.google.com/github/CarolWuuu/FinanceAPI/blob/main/FinanceAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import requests
import pandas as pd
import time

Task 1: Fetch Stock Data

In [84]:


def get_stock_data(symbols, api_key):
    url = 'https://yfapi.net/v6/finance/quote'
    querystring = {"symbols": symbols}

    headers = {
        'x-api-key': api_key,
    }

    response = requests.request("GET",url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()

        if 'quoteResponse' in data and 'result' in data['quoteResponse']:
            for stock in data['quoteResponse']['result']:
                print(f"Stock Ticker: {stock['symbol']}, Company: {stock['longName']}, Current Market Price: ${stock['regularMarketPrice']}")
        else:
            print("Error: No stock data found.")
    else:
        print(f"Error: Unable to fetch data (status code: {response.status_code}).")

if __name__ == "__main__":
    api_key = input("Enter your API key: ")
    symbols = input("Enter stock symbols (comma-separated): ")

    get_stock_data(symbols, api_key)


Enter your API key: bdiRYDtgLj4mnPdtbqJuw3iQCRnBS9wB7xgUb5g3 
Enter stock symbols (comma-separated): AAPL
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $247.97


Task 2.1: Fetch Additional Data

In [87]:

def get_module(symbol, module, api_key):
    url=f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
    query_additional = {"symbol": symbol, "modules": module}
    headers = {
        'x-api-key': api_key,
    }
    response = requests.request("GET",url, headers=headers, params=query_additional)

    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data["quoteSummary"]["result"])
        display(df)

    else:
        print(f"Error: Unable to fetch data (status code: {response.status_code}).")


def get_stock_metrics(symbol, api_key):
    url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
    params = {"modules": "summaryDetail,financialData"}
    headers = {"x-api-key": api_key}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        result = data.get("quoteSummary", {}).get("result", [{}])[0]

        high_52w = result.get("summaryDetail", {}).get("fiftyTwoWeekHigh", {}).get("raw","N/A")
        low_52w = result.get("summaryDetail", {}).get("fiftyTwoWeekLow", {}).get("raw","N/A")
        roa = result.get("financialData", {}).get("returnOnAssets", {}).get("raw","N/A")

        return high_52w, low_52w, roa
    else:
        print(f"Error: Unable to fetch data (status code: {response.status_code})")
        return "N/A","N/A","N/A"


if __name__ == "__main__":
    api_key = input("Enter your API key: ")
    symbol = input("Enter a single stock symbol: ")
    module = input("Enter a single module: ")

    get_module(symbol, module, api_key)
    high_52w, low_52w, roa=get_stock_metrics(symbol,api_key)
    print(f"52 Week High: {high_52w}, 52 Week Low: {low_52w}, Return on Assets (ROA): {roa}")




Enter your API key: bdiRYDtgLj4mnPdtbqJuw3iQCRnBS9wB7xgUb5g3
Enter a single stock symbol: AAPL
Enter a single module: price


,price.maxAge,price.preMarketChangePercent.raw,price.preMarketChangePercent.fmt,price.preMarketChange.raw,price.preMarketChange.fmt,price.preMarketTime,price.preMarketPrice.raw,price.preMarketPrice.fmt,price.preMarketSource,price.regularMarketChangePercent.raw,...,price.longName,price.currency,price.quoteSourceName,price.currencySymbol,price.fromCurrency,price.toCurrency,price.lastMarket,price.marketCap.raw,price.marketCap.fmt,price.marketCap.longFmt
0,1,0.003399,0.34%,0.839996,0.84,1740493799,247.94,247.94,FREE_REALTIME,0.004452,...,Apple Inc.,USD,Nasdaq Real Time Price,$,None,None,None,3728485384192,3.73T,"3,728,485,384,192.00"


52 Week High: 260.1, 52 Week Low: 164.08, Return on Assets (ROA): 0.22518998


Task 2.2

In [88]:
def get_trending_stocks(region, api_key):
    url_trending=f"https://yfapi.net/v1/finance/trending/{region}"
    query_trending = {"region":region}
    headers = {
        'x-api-key': api_key,
    }
    response_trending = requests.request("GET",url_trending, headers=headers, params=query_trending)

    if response_trending.status_code == 200:
        data_trending = response_trending.json()
        stocks = [quote["symbol"] for quote in data_trending["finance"]["result"][0]["quotes"]]

        trending_list = []
        for stock in stocks:
            time.sleep(1)
            url_quote = 'https://yfapi.net/v6/finance/quote'
            query_quote = {"symbols": stock}
            response_quote = requests.request("GET",url_quote, headers=headers, params=query_quote)
            data_quote = response_quote.json()

            high_52w, low_52w, roa = get_stock_metrics(stock, api_key)

            for stock in data_quote['quoteResponse']['result']:
              trending_list.append({
                  "Stock Ticker": stock['symbol'],
                  "Company": stock['longName'],
                  "Current Price": f"${stock['regularMarketPrice']}",
                  "52 Week High": f"${high_52w}",
                  "52 Week Low": f"${low_52w}"
              })

        df = pd.DataFrame(trending_list)
        print(df)

    else:
        print(f"Error: Unable to fetch data (status code: {response_trending.status_code}).")

if __name__ == "__main__":
    region = input("Enter a region: ")
    api_key = input("Enter your API key: ")
    get_trending_stocks(region, api_key)

Enter a region: US
Enter your API key: bdiRYDtgLj4mnPdtbqJuw3iQCRnBS9wB7xgUb5g3
   Stock Ticker                         Company Current Price 52 Week High  \
0          TSLA                     Tesla, Inc.       $302.43      $488.54   
1       BTC-USD                     Bitcoin USD    $87725.086   $109114.88   
2          ONVO         Organovo Holdings, Inc.         $1.25        $1.83   
3            HD            The Home Depot, Inc.        $394.6      $439.37   
4          SMCI      Super Micro Computer, Inc.         $46.9       $122.9   
5         ^IXIC                NASDAQ Composite    $19102.998    $20204.58   
6            LI                    Li Auto Inc.       $29.965       $46.44   
7           SRE                          Sempra        $70.71       $95.77   
8          WAFU  Wah Fu Education Group Limited        $3.235      $7.4899   
9            WU       The Western Union Company         $10.9       $14.19   
10         NVDA              NVIDIA Corporation      $128.675 